In [ ]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython import get_ipython
ipython = get_ipython()
ipython.run_line_magic('config', "InlineBackend.figure_formats = {'png', 'pdf'}")

# Control Of Idealized Delay+Integrator Plant
This model responds as a delay+integrator at a fixed update rate.

In [ ]:
from PIFilter import IIR_PI_Filter, calculate_pi_coefficients
from delay_integrator_model import DelayIntegratorPlantModel
from matplotlib import pyplot as plt

In [ ]:
p = DelayIntegratorPlantModel(delay=1, gain=1, rate=1000, ambient=293, temperature=293, heat_leak=1)
p.reset()

## Plant+ Heat Leak
The plant model includes a heat leak which is the same units as the gain. A heat leak equal to the control setting will result in a constant temperature.

Show the plant cooling to the ambient temperature.

In [ ]:
p.reset()
p.temperature = 400
d = []
for _ in range(10*p.delay*p.rate):
    d.append(p.update(0))

fig, ax = plt.subplots()
ax.plot(d)
ax.set_title("Ambient Cooling of Object Due To Heat Leak")
ax.set_xlabel("Sample Number")
ax.set_ylabel("Temperature (k)")
plt.show()

## Constant Set Point
Show the plant reaching a constant temperature with different control settings. The difference will be in the temperature difference the plant rests at.

In [ ]:
fig, ax = plt.subplots()

for set_point in range(5):
    d = []
    p.reset()
    p.temperature = p.ambient
    for _ in range(10*p.delay*p.rate):
        d.append(p.update(set_point))
    
    ax.plot(d, label="Set Point = %d"%set_point)
ax.set_title("Plant With Constant Control Input, Heating From Ambient With Heat Leak")
ax.set_xlabel("Sample Number")
ax.set_ylabel("Temperature (k)")
plt.legend()
plt.show()

## Idealized PI Filter
Introduce an idealized filter with no output limits. Remove the heat leak to get the ideal response. To start use the SMIC tuning algorithm (https://folk.ntnu.no/skoge/publications/2012/skogestad-improved-simc-pid/old-submitted/simcpid.pdf).

In [ ]:
set_point = 300
p = DelayIntegratorPlantModel(delay=1, gain=1, rate=1000, ambient=293, heat_leak=0)
kp, ki = calculate_pi_coefficients(p.delay, p.gain, update_frequency=p.rate)
ki = ki *p.rate
f = IIR_PI_Filter(kp=kp, ki=ki, sample_time=1/p.rate, set_point=set_point)


temp = []
integral = []
command = []

p.reset()
p.temperature = p.ambient
for _ in range(50*p.delay*p.rate):
    control = f.update(p.temperature)
    command.append(control)
    integral.append(f.integral)
    temp.append(p.update(control))

fig, ax = plt.subplots()
ax.plot(temp, label="Temperature")
ax.plot([0, len(temp)], [set_point]*2, label="Set Point")
ax.set_xlabel("Sample Number")
ax.set_ylabel("Temperature (k)")
plt.legend()
plt.show()

fig, ax = plt.subplots()
ax.plot(integral[2:], label="Integral")
ax.set_xlabel("Sample Number")
ax.set_ylabel("Integral Value")
plt.legend()
plt.show()

fig, ax = plt.subplots()
ax.plot(command, label="Control")
ax.set_xlabel("Sample Number")
ax.set_ylabel("Control Setting")
plt.legend()
plt.show()

## Limited Output
The controller usually does not have unlimited current output so we need to introduce limits.
The basic implimentation just caps the output at the limit. This will cause windup of the integrator which is shown.

## Integrator Windup
With the limited output the integrator windsup and causes bad behaviour.

In [ ]:
set_point = 300
f = IIR_PI_Filter(kp=kp, ki=ki, sample_time=1/p.rate, set_point=set_point, ilim=0.1, antiwindup=True)

temp = []
integral = []
command = []

p.reset()
p.temperature = p.ambient
for _ in range(500*p.delay*p.rate):
    control = f.update(p.temperature)
    command.append(control)
    integral.append(f.integral)
    temp.append(p.update(control))

fig, ax = plt.subplots()
ax.plot(temp, label="Temperature")
ax.plot([0, len(temp)], [set_point]*2, label="Set Point")
ax.set_xlabel("Sample Number")
ax.set_ylabel("Temperature (k)")
plt.legend()
plt.show()

fig, ax = plt.subplots()
ax.plot(integral, label="Integral")
ax.set_xlabel("Sample Number")
ax.set_ylabel("Integral Value")
plt.legend()
plt.show()

fig, ax = plt.subplots()
ax.plot(command, label="Control")
ax.set_xlabel("Sample Number")
ax.set_ylabel("Control Setting")
plt.legend()
plt.show()

## Anti-windup Controller
To prevent windup there are several methods that can be used of increasing complexity. The simpliest is freezing the integrator value at the point the control setting is beyond the controllers capability.

Freezing the integrator leads to less overshoot than the normal state and none of the ringing occurs.

In [ ]:
set_point = 300
f = IIR_PI_Filter(kp=kp, ki=ki, sample_time=1/p.rate, set_point=set_point, ilim=0.1, antiwindup=True)

temp = []
integral = []
command = []

p.reset()
p.temperature = p.ambient
for _ in range(200*p.delay*p.rate):
    control = f.update(p.temperature)
    command.append(control)
    integral.append(f.integral)
    temp.append(p.update(control))

fig, ax = plt.subplots()
ax.plot(temp, label="Temperature")
ax.set_title("Temperature")
ax.plot([0, len(temp)], [set_point]*2, label="Set Point")
ax.set_xlabel("Sample Number")
ax.set_ylabel("Temperature (k)")
plt.legend()
plt.show()

fig, ax = plt.subplots()
ax.plot(integral, label="Integral")
ax.set_title("Integral")
ax.set_xlabel("Sample Number")
ax.set_ylabel("Integral Value")
plt.legend()
plt.show()

fig, ax = plt.subplots()
ax.plot(command, label="Control")
ax.set_title("Control Setting")
ax.set_xlabel("Sample Number")
ax.set_ylabel("Control Setting")
plt.legend()
plt.show()